## I am part of a team for building a pipeline which will predict the next word which will be used by a social media application called SNIPCHAT. As a data scientist, I will build a model using Recurrent Neural Network and will train and test the model which can be used in the application in the future.
First I will import all the necessary libraries, then load the dataset which is a novel called sherlock holmes.
After that i will clean the dataset, remove unnecessary characters and then tokenize it. Then i will do the feature engineering part. After that I will use Bidirectional LSTM model. After buildig the model,  I will train the model with 20 epochs.And then test the model for next word predcition.



https://thecleverprogrammer.com/2020/07/20/next-word-prediction-model/

https://www.kaggle.com/code/ysthehurricane/next-word-prediction-bi-lstm-tutorial-easy-way/data - DATASET



## IMPORTING LIBRARIES

In [2]:
import pandas as pd
import os
import numpy as np
import pickle
import heapq
import re
import nltk
import tensorflow as tf
import sklearn.feature_extraction.text
import sklearn.metrics.pairwise
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from nltk.tokenize import RegexpTokenizer
from tensorflow.python.keras.layers.core import Dense, Activation
from tensorflow.keras.optimizers import RMSprop


2022-12-22 11:28:24.550548: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-22 11:28:24.550576: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


### LOADING DATASET

In [3]:
df = pd.read_csv("../../datasets/data.csv")
df.head()

,id,url,title,subtitle,image,claps,responses,reading_time,publication,date
0,1,https://towardsdatascience.com/a-beginners-gui...,A Beginner’s Guide to Word Embedding with Gens...,NaN,1.png,850,8,8,Towards Data Science,2019-05-30
1,2,https://towardsdatascience.com/hands-on-graph-...,Hands-on Graph Neural Networks with PyTorch & ...,NaN,2.png,1100,11,9,Towards Data Science,2019-05-30
2,3,https://towardsdatascience.com/how-to-use-ggpl...,How to Use ggplot2 in Python,A Grammar of Graphics for Python,3.png,767,1,5,Towards Data Science,2019-05-30
3,4,https://towardsdatascience.com/databricks-how-...,Databricks: How to Save Files in CSV on Your L...,When I work on Python projects dealing…,4.jpeg,354,0,4,Towards Data Science,2019-05-30
4,5,https://towardsdatascience.com/a-step-by-step-...,A Step-by-Step Implementation of Gradient Desc...,One example of building neural…,5.jpeg,211,3,4,Towards Data Science,2019-05-30


In [4]:
print("records : ",df.shape[0])
print("fields : ",df.shape[1])

records :  6508
fields :  10


We have 6508 number of records and 10 fields. But I will only use title coloumn for predicting next word.

## PRINTING COLUMN NAMES

In [5]:
print(df.columns)

Index(['id', 'url', 'title', 'subtitle', 'image', 'claps', 'responses',
       'reading_time', 'publication', 'date'],
      dtype='object')


## DISPALYING TITLE OF ARTICLES

In [6]:
df["title"]

0       A Beginner’s Guide to Word Embedding with Gens...
1       Hands-on Graph Neural Networks with PyTorch & ...
2                            How to Use ggplot2 in Python
3       Databricks: How to Save Files in CSV on Your L...
4       A Step-by-Step Implementation of Gradient Desc...
                              ...                        
6503    “We” vs “I” — How Should You Talk About Yourse...
6504                     How Donald Trump Markets Himself
6505        Content and Marketing Beyond Mass Consumption
6506    5 Questions All Copywriters Should Ask Clients...
6507               How To Write a Good Business Blog Post
Name: title, Length: 6508, dtype: object

## REMOVING UNWANTED CHARACTERS AND WORDS

Since there are many unwanted characters and words, there might be a chance that these can decrease the efficiency of the model. So I am going to clean the data.

In [7]:
df["title"] = df["title"].apply(lambda x: x.replace(u'\xa0',u' '))
df["title"] = df["title"].apply(lambda x: x.replace('\u200a ',''))



## TOKENIZATION

Next I am going to tokeize the text to give unique to all the words to make it into vocabulary. 

In [8]:
tokenizer = Tokenizer(oov_token='<oov>')  #words which cannot be obtained in word_index
tokenizer.fit_on_texts(df['title'])
total_words = len(tokenizer.word_index) + 1
    

## BAG OF NGRAM MODEL

Now I am going to convert titles to sequences and then use the n gram model to do good predcition.

In [9]:
input_sequences = []
for l in df['title']:
    token = tokenizer.texts_to_sequences([l])[0]
   
    
    for i in range(1, len(token)):
        n_gram = token[:i+1]
        input_sequences.append(n_gram)


print("Total input sequences: ", len(input_sequences))

Total input sequences:  47594


here I have displayed the total input sequence of words whcih is 47594

## make all the titles into same length

So to make all the titles into same length I will be using padding to do that process. That is I will find the title with maximum length and then pad other titles accordingly

In [10]:
max_sequence = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence, padding='pre'))


Here maxlen is used to get maximum line length and padding is set to pre because we want to make sure the hidden state output is correct. If we use post-padding, final hidden state will be eliminated so it would be mostly [0]

## Splitting feature and labels

Here I will define some functions for storing features and their corresponding labels. For that I will consider the last elemenst of the sequence as labels and then use onehot encoding on the labels to convert categorical variables as binary vectors.

In [11]:
x,lab = input_sequences[:,:-1], input_sequences[:,-1]
y = tf.keras.utils.to_categorical(lab, num_classes = total_words)

In [12]:
print(x[3])
print(y[3][10])
print(lab[3])

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   5 665  65
   2]
0.0
448


here i have stores features and labels and have displayed them as arrays.

## Training the model

Now I am going to use Bidirectional LSTM  to train the model.

In [13]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence-1))
model.add(Bidirectional(LSTM(150)))
model.add(Dense(total_words, activation='softmax'))
adam = Adam(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
history = model.fit(x, y, epochs=20, verbose=1)
print(model)

2022-12-22 11:28:33.403016: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-12-22 11:28:33.403052: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-22 11:28:33.403083: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (Ubuntu-1804-bionic-64-minimal): /proc/driver/nvidia/version does not exist
2022-12-22 11:28:33.403370: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/lib/python3.8/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:3

Epoch 1/20
1488/1488 [==============================] - 54s 35ms/step - loss: 6.8135 - accuracy: 0.1261
Epoch 2/20
1488/1488 [==============================] - 52s 35ms/step - loss: 5.7907 - accuracy: 0.1784
Epoch 3/20
1488/1488 [==============================] - 52s 35ms/step - loss: 4.9145 - accuracy: 0.2102
Epoch 4/20
1488/1488 [==============================] - 52s 35ms/step - loss: 4.1662 - accuracy: 0.2551
Epoch 5/20
1488/1488 [==============================] - 52s 35ms/step - loss: 3.6061 - accuracy: 0.3064
Epoch 6/20
1488/1488 [==============================] - 52s 35ms/step - loss: 3.2477 - accuracy: 0.3495
Epoch 7/20
1488/1488 [==============================] - 52s 35ms/step - loss: 2.9967 - accuracy: 0.3781
Epoch 8/20
1488/1488 [==============================] - 52s 35ms/step - loss: 2.8323 - accuracy: 0.4007
Epoch 9/20
1488/1488 [==============================] - 52s 35ms/step - loss: 2.6707 - accuracy: 0.4246
Epoch 10/20
1488/1488 [==============================] - 52s 35m

Here I have added embedding layer and dense layer to the model. As paramters I used size of the vocabulary,dimension of dense layer and set activation as softmax so that the activation for the last layer of a classification network could be interpreted as a probability distribution. And used adam as optimizer, I also displayed the accuracy of eah epochs till 20 of the validation. Since there are 2 or more output lables, I have used categorical_crossentropy as loass fucntion for the multi class classification model. And I set verbose as 1 just to show how my output sould look like, I can aslo set it as 0,2 or 3. And for adam I set the learning rate as 0.01 which depends on how fast or slow the model trains. 

## TESTING THE MODEL FOR PREDICTION

In [14]:
seed_text = "happy and"
next_words = 3
  
for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence-1, padding='pre')
    predict = model.predict_classes(token_list, verbose=0)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predict:
            output_word = word
            break
    seed_text += " " + output_word
print(seed_text)

/usr/local/lib/python3.8/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


happy and rejection is the


Here the seed_text and next_words can be changed according to the prediction process. Currently I set seed_text as "use of" and next_words to 5 and the ouput I got is "use of the stars episode i use". If I change the seed_text and next_words to "happy and" and 3 respectively the predicted output is "happy and rejection is the". 

The model can be used for not only app but for many search engines like google, brave etc. And with minor upgrades, the model holds many future opportunities for prediction.